# Regressions

In [39]:
library(dplyr)
library(fixest)
library(MASS)

library(stargazer)


Attaching package: 'MASS'


The following object is masked from 'package:dplyr':

    select




# Loading the Data

In [7]:
dataset <- read.csv('../../Data/Data Sets/cleaned_dataset.csv')

merseyside <- dataset %>% filter(PoliceDept == "Merseyside")
london <- dataset %>% filter(PoliceDept == "London")

head(merseyside)
head(london)

,LSOA21CD,Borough,StopCount_z,StopCount,PoliceDept,MeanHousePrice_z,CrimeSum_z,BAME_z,DrugCrimeSum_z,IncomeDomainScore_z
,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,E01006434,Knowsley,0.634,132,Merseyside,-0.769,0.151,-0.673,0.382,-1.087
2,E01006435,Knowsley,0.446,109,Merseyside,-0.660,-0.146,-0.505,0.204,-1.102
3,E01006436,Knowsley,2.047,305,Merseyside,-0.893,0.449,-0.512,1.629,-1.119
4,E01006437,Knowsley,4.774,639,Merseyside,-0.813,2.759,-0.542,2.128,-1.015
5,E01006438,Knowsley,-0.133,38,Merseyside,1.342,-0.364,-0.279,-0.188,0.605
6,E01006439,Knowsley,-0.239,25,Merseyside,0.415,-0.131,-0.551,-0.330,-0.581


,LSOA21CD,Borough,StopCount_z,StopCount,PoliceDept,MeanHousePrice_z,CrimeSum_z,BAME_z,DrugCrimeSum_z,IncomeDomainScore_z
,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,E01000001,City of London,-0.095,25,London,0.580,-0.564,-1.353,-0.435,2.193
2,E01000002,City of London,-0.394,2,London,0.749,-0.484,-1.251,-0.435,1.851
3,E01000003,City of London,-0.368,4,London,-0.176,-0.619,-1.033,-0.435,0.494
4,E01000005,City of London,0.230,50,London,0.000,-0.174,0.871,-0.251,-1.042
5,E01032739,City of London,0.543,74,London,0.805,2.033,-0.693,0.545,2.171
6,E01032740,City of London,-0.251,13,London,0.525,0.539,-0.806,0.055,2.071


# Model Selection

**choosing between:**
- Linear Regression
- Poisson Regression
- negative binomial

(all fixed effects on borough)

In [ ]:
# normal poisson model
poisson_model <- glm(StopCount ~ IncomeDomainScore_z + MeanHousePrice_z + CrimeSum_z + BAME_z + DrugCrimeSum_z,
                     data = london, 
                     family = poisson)

negbin_model <- glm.nb(StopCount ~ IncomeDomainScore_z + MeanHousePrice_z + CrimeSum_z + BAME_z + DrugCrimeSum_z,
                     data = london)

#Warning message:
"glm.fit: algorithm did not converge"

Warning message:
"glm.fit: algorithm did not converge"


In [35]:
poisson_model <- fepois(StopCount ~ IncomeDomainScore_z + MeanHousePrice_z + CrimeSum_z + BAME_z + DrugCrimeSum_z | Borough,
                        data = london)

negbin_model <- fenegbin(StopCount ~ IncomeDomainScore_z + MeanHousePrice_z + CrimeSum_z + BAME_z + DrugCrimeSum_z | Borough,
                        data = london)

In [41]:
summary(poisson_model)
summary(negbin_model)


Call:
glm(formula = StopCount ~ IncomeDomainScore_z + MeanHousePrice_z + 
    CrimeSum_z + BAME_z + DrugCrimeSum_z, family = poisson, data = london)

Coefficients:
                     Estimate Std. Error z value Pr(>|z|)    
(Intercept)          3.344667   0.002760 1211.91   <2e-16 ***
IncomeDomainScore_z -0.234039   0.003330  -70.27   <2e-16 ***
MeanHousePrice_z     0.095383   0.002338   40.80   <2e-16 ***
CrimeSum_z           0.177904   0.001602  111.07   <2e-16 ***
BAME_z               0.220871   0.002991   73.86   <2e-16 ***
DrugCrimeSum_z      -0.027757   0.001360  -20.40   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 287836  on 4993  degrees of freedom
Residual deviance: 206475  on 4988  degrees of freedom
AIC: 228422

Number of Fisher Scoring iterations: 6



Call:
glm.nb(formula = StopCount ~ IncomeDomainScore_z + MeanHousePrice_z + 
    CrimeSum_z + BAME_z + DrugCrimeSum_z, data = london, init.theta = 0.957039931, 
    link = log)

Coefficients:
                    Estimate Std. Error z value Pr(>|z|)    
(Intercept)          3.19269    0.01480 215.730  < 2e-16 ***
IncomeDomainScore_z -0.19577    0.01903 -10.288  < 2e-16 ***
MeanHousePrice_z     0.07320    0.01646   4.447 8.72e-06 ***
CrimeSum_z           0.23522    0.02825   8.326  < 2e-16 ***
BAME_z               0.20955    0.01801  11.633  < 2e-16 ***
DrugCrimeSum_z       0.58703    0.02802  20.947  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Negative Binomial(0.957) family taken to be 1)

    Null deviance: 8327.4  on 4993  degrees of freedom
Residual deviance: 5701.3  on 4988  degrees of freedom
AIC: 42117

Number of Fisher Scoring iterations: 1


              Theta:  0.9570 
          Std. Err.:  0.0186 

 2 x log-likel

In [34]:
AIC(ols_model, poisson_model, negbin_model)
BIC(ols_model, poisson_model, negbin_model)

[1]   9896.081 214098.843  41751.333

[1]  10143.69 214346.45  41998.94

In [37]:
library(AER)
dispersiontest(poisson_model)


Warning message:
"package 'AER' was built under R version 4.3.3"
Loading required package: car

Warning message:
"package 'car' was built under R version 4.3.3"
Loading required package: carData

Warning message:
"package 'carData' was built under R version 4.3.3"

Attaching package: 'car'


The following object is masked from 'package:dplyr':

    recode


Loading required package: lmtest

Warning message:
"package 'lmtest' was built under R version 4.3.3"
Loading required package: zoo

Warning message:
"package 'zoo' was built under R version 4.3.3"

Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Loading required package: sandwich

Warning message:
"package 'sandwich' was built under R version 4.3.3"
Loading required package: survival



ERROR: Error in dispersiontest(poisson_model): only Poisson GLMs can be tested


In [ ]:
mean(merseyside$StopCount)
var(merseyside$StopCount)


AIC(poisson_model, negbin_model)


[1] 54.31419

[1] 15002.14

[1] -0.443

[1] 214098.84  41751.33

In [3]:
# Load necessary library
library(fixest)

# Fit a fixed effects model with 'LSOA21CD' as the fixed effect
model <- feols(StopCount_z ~ IncomeDomainScore + MeanHousePrice_z + CrimeSum_z + BAME_z + DrugCrimeSum_z | Borough, 
data = merseyside)

# Summarise the model
summary(model)


Warning message:
"package 'fixest' was built under R version 4.3.3"


OLS estimation, Dep. Var.: StopCount_z
Observations: 923
Fixed-effects: Borough: 5
Standard-errors: Clustered (Borough) 
                       Estimate Std. Error   t value   Pr(>|t|)    
IncomeDomainScore  0.0000000898 0.00000158  0.056641 9.5755e-01    
MeanHousePrice_z   0.0027600209 0.00914901  0.301674 7.7793e-01    
CrimeSum_z        -0.1109387020 0.06352800 -1.746296 1.5569e-01    
BAME_z             0.0737369356 0.00931544  7.915565 1.3784e-03 ** 
DrugCrimeSum_z     1.0009077971 0.04883804 20.494431 3.3477e-05 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
RMSE: 0.391514     Adj. R2: 0.845032
                 Within R2: 0.840916

In [28]:
library(fixest)

# Use the actual count, not the z-score, as outcome
model_pois <- fepois(StopCount ~ IncomeDomainScore + MeanHousePrice_z + CrimeSum_z + BAME_z + DrugCrimeSum_z | Borough,
                     data = merseyside)

summary(model_pois)


Poisson estimation, Dep. Var.: StopCount
Observations: 923
Fixed-effects: Borough: 5
Standard-errors: Clustered (Borough) 
                   Estimate Std. Error   z value   Pr(>|z|)    
IncomeDomainScore -0.000037   0.000022 -1.651806 9.8574e-02 .  
MeanHousePrice_z  -0.323193   0.149856 -2.156688 3.1030e-02 *  
CrimeSum_z        -0.007119   0.069291 -0.102739 9.1817e-01    
BAME_z             0.188839   0.017336 10.892705  < 2.2e-16 ***
DrugCrimeSum_z     0.268332   0.036750  7.301580 2.8441e-13 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Log-Likelihood: -21,772.2   Adj. Pseudo R2: 0.558184
           BIC:  43,612.7     Squared Cor.: 0.623977

In [6]:
model_nb <- fenegbin(StopCount ~ IncomeDomainScore + MeanHousePrice_z + CrimeSum_z + BAME_z + DrugCrimeSum_z | Borough,
                     data = merseyside)

summary(model_nb)

model_nb <- fenegbin(StopCount ~ IncomeDomainScore + MeanHousePrice_z + CrimeSum_z + BAME_z + DrugCrimeSum_z | Borough,
                     data = london)

summary(model_nb)


ML estimation, family = Negative Binomial, Dep. Var.: StopCount
Observations: 923
Fixed-effects: Borough: 5
Standard-errors: Clustered (Borough) 
                   Estimate Std. Error  z value   Pr(>|z|)    
IncomeDomainScore -0.000044 0.00000937 -4.74310 2.1047e-06 ***
MeanHousePrice_z  -0.064860 0.04420109 -1.46738 1.4227e-01    
CrimeSum_z        -0.185603 0.17468895 -1.06248 2.8802e-01    
BAME_z             0.420807 0.07051161  5.96790 2.4032e-09 ***
DrugCrimeSum_z     0.894557 0.25151928  3.55661 3.7567e-04 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Over-dispersion parameter: theta = 1.352511  
Log-Likelihood: -4,115.3   Adj. Pseudo R2: 0.091695
           BIC:  8,298.8     Squared Cor.: 0.320655

ML estimation, family = Negative Binomial, Dep. Var.: StopCount
Observations: 4,994
Fixed-effects: Borough: 33
Standard-errors: Clustered (Borough) 
                   Estimate Std. Error   z value   Pr(>|z|)    
IncomeDomainScore -0.000011 0.00000460 -2.415931 1.5695e-02 *  
MeanHousePrice_z  -0.011520 0.04262145 -0.270290 7.8694e-01    
CrimeSum_z         0.253538 0.07384298  3.433474 5.9590e-04 ***
BAME_z             0.277677 0.04801988  5.782546 7.3578e-09 ***
DrugCrimeSum_z     0.537849 0.05285899 10.175169  < 2.2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Over-dispersion parameter: theta = 1.034285  
Log-Likelihood: -20,837.7   Adj. Pseudo R2: 0.05706 
           BIC:  41,998.9     Squared Cor.: 0.177017

In [7]:
model_nb <- fenegbin(StopCount ~ IncomeDomainScore + MeanHousePrice_z + CrimeSum_z + BAME_z + DrugCrimeSum_z + BAME_z*IncomeDomainScore| Borough,
                     data = merseyside)

summary(model_nb)

model_nb <- fenegbin(StopCount ~ IncomeDomainScore + MeanHousePrice_z + CrimeSum_z + BAME_z + DrugCrimeSum_z + BAME_z*IncomeDomainScore| Borough,
                     data = london)

summary(model_nb)

ML estimation, family = Negative Binomial, Dep. Var.: StopCount
Observations: 923
Fixed-effects: Borough: 5
Standard-errors: Clustered (Borough) 
                          Estimate Std. Error  z value   Pr(>|z|)    
IncomeDomainScore        -0.000039 0.00001809 -2.13139 0.03305725 *  
MeanHousePrice_z         -0.054772 0.03561125 -1.53804 0.12403851    
CrimeSum_z               -0.189323 0.16127656 -1.17390 0.24043372    
BAME_z                    0.174682 0.01888372  9.25042  < 2.2e-16 ***
DrugCrimeSum_z            0.944381 0.28748308  3.28500 0.00101983 ** 
IncomeDomainScore:BAME_z  0.000030 0.00000773  3.84446 0.00012082 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Over-dispersion parameter: theta = 1.414544  
Log-Likelihood: -4,094.5   Adj. Pseudo R2: 0.09606 
           BIC:  8,264.0     Squared Cor.: 0.316299

ML estimation, family = Negative Binomial, Dep. Var.: StopCount
Observations: 4,994
Fixed-effects: Borough: 33
Standard-errors: Clustered (Borough) 
                            Estimate Std. Error   z value   Pr(>|z|)    
IncomeDomainScore        -0.00000616 0.00000483 -1.274457 0.20250140    
MeanHousePrice_z         -0.00068233 0.04426585 -0.015414 0.98770157    
CrimeSum_z                0.24711499 0.07696828  3.210608 0.00132454 ** 
BAME_z                    0.07241292 0.05999964  1.206889 0.22747479    
DrugCrimeSum_z            0.57440044 0.05580889 10.292276  < 2.2e-16 ***
IncomeDomainScore:BAME_z  0.00001438 0.00000377  3.813774 0.00013686 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Over-dispersion parameter: theta = 1.043701  
Log-Likelihood: -20,813.3   Adj. Pseudo R2: 0.058117
           BIC:  41,958.7     Squared Cor.: 0.177013

stop counts increase more sharply in deprived areas where the BAME population is higher.